In [5]:
import numpy as np
import numpy.testing as test
import scipy.linalg as LA
#

In [34]:
def getmat(G, A,y,mu):
    """requires: G, A,y,mu"""
    assert(len(y.shape)==1),"require 1d arrays"
    m,n=A.shape
    #cero1=np.zeros((n,m))
    row1=np.hstack((G,np.zeros((n,m)),A.T))
    row2=np.hstack((A,np.eye(m),np.zeros((m,m))))
    row3=np.hstack((np.zeros((m,n)),np.diag(mu),np.diag(y)))

    mat=np.vstack((row1,row2,row3))
    return mat


In [35]:
A=np.ones((2,3))
G=np.eye(3)
y=2*np.ones(2)
mu=3*np.ones(2)

getmat(G,A,y,mu)

array([[1., 0., 0., 0., 0., 1., 1.],
       [0., 1., 0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 0., 1., 1.],
       [1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 1., 0., 0.],
       [0., 0., 0., 3., 0., 2., 0.],
       [0., 0., 0., 0., 3., 0., 2.]])

In [9]:
def F(x,y,mu,prob):
    """requires: G, A,b,c"""
    G,A,b,c=prob["G"],prob["A"],prob["b"],prob["c"]
    assert(len(x.shape)==1),"requires 1d arrays"

    row1=G@x+A.T@mu+c
    row2=A@x+y-b
    row3=y*mu

    Fval=np.hstack((row1,row2,row3))

    return Fval
    

In [13]:
row1=np.ones(4)
row2=np.ones(3)
row3=np.ones(3)
np.hstack((row1,row2,row3))

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [28]:
def recorte(v,dv):
    #use mask
    #vi+alpha+dvi>=0
    mask=dv<0
    if any(mask):
        alpha=1
        for i in range(len(v)):
            if mask[i]:
                alpha=min(alpha,-v[i]/dv[i])
    else:
        alpha=1
    
    return alpha

In [33]:
#Prueba recorte
v=np.random.rand(5)
dv=np.random.randn(5)
print(v,dv)
alpha=recorte(v,dv)
v+0.99*alpha*dv

[0.23608342 0.82997342 0.35944529 0.05771671 0.01149595] [-1.50163033 -0.55530541 -0.10061887  0.50883011 -1.25180339]


array([2.22431075e-01, 8.24924761e-01, 3.58530492e-01, 6.23428258e-02,
       1.14959542e-04])

In [ ]:
def mNewton(x0,y0,mu0,prob,tol=1e-6,itmax=400):
    x,y,mu=x0.copy(),y0.copy(),mu0.copy()
    Fk=F(x,y,mu,prob)
    n,m=len(x),len(y)
    sigma=0.1
    for i in range(itmax):
        if LA.norm(Fk,np.inf)<= tol:
            break   
        mat=getmat(G, A,y,mu)
        Mk=y.T@mu/m
        vec=np.hstack((np.zeros(n+m),sigma*Mk*np.ones(m)))

        deltas=LA.solve(mat,-Fk+vec)
        dx=deltas[:n]
        dy=deltas[n:n+m]
        dmu=deltas[n+m:]
        assert(len(dy)== m),"lenght of y wrong"
        alphay = recorte(y,dy)
        alphamu = recorte(mu,dmu)
        alpha = min(alphay,alphamu)
        x+=alpha*dx
        y+=alpha*dy
        mu+=alpha*dmu

    return (x,y,mu)
        


   